# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

これまで、RNN（やLSTM）を用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できるのではないか。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプション生成などに応用できそう。隠れ状態を通じてRNNからCNNまで逆伝播が繋がるので、画像と文章のペアさえ用意すれば学習できそう。というかできる[1]。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。

本章ではこの翻訳モデルを作成する。入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。こういった、シーケンスをシーケンスに変換するモデルを *Seq2Seq (Sequence to Sequence)* と呼ぶ。

Seq2Seqは以下の二つのRNNから構成される。

- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings; warnings.filterwarnings("ignore")
import math
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットから日本語と英語の対訳コーパスを使用する。

- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    "huggingface:iwslt2017/iwslt2017-en-ja",
    data_dir="data",
    split="train"
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample["translation"]["ja"].decode()
    en = sample["translation"]["en"].decode()
    data_ja.append(ja)
    data_en.append(en)

print("num of data:", len(data_ja), "\n")
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

腐敗が 病的に民主主義を 破壊してしまうということです つまり どのようなシステムであっても その構成員がごく少数の 構成員によって選ばれる場合には その構成員がごく少数の 構成員によって選ばれる場合には それは ごく少数の構成員が ごくごく少数の構成員が 改革を阻止できることを意味します
It's a pathological, democracy-destroying corruption, because in any system where the members are dependent upon the tiniest fraction of us for their election, that means the tiniest number of us, the tiniest, tiniest number of us, can block reform.

これは大変なことです
This is a big deal.

疑問というのは 西洋社会におけるすべての議論は 課税レベルに関することです
We ask the question -- the whole debate in the Western world is about the level of taxation.

つまり大きな意味では 技術というのは何も こんな機器だけをさすのではないのです 習慣、テクニック、心理的手法 なども含めて 技術と呼べるのです
So in a broad sense, we don't need to think about technology as only little gadgets, like these things here, but even institutions and techniques, psychological methods and so forth.

ひも理論が難しいなんて人もいますが 楽なもんです
And some people think that string theory is tough.



書き出し

In [5]:
textfile_ja = "data/iwslt2017_ja.txt"
with open(textfile_ja, "w") as f:
    f.write("\n".join(data_ja))

textfile_en = "data/iwslt2017_en.txt"
with open(textfile_en, "w") as f:
    f.write("\n".join(data_en))

読み込み

In [3]:
textfile_ja = f"data/iwslt2017_ja.txt"
textfile_en = f"data/iwslt2017_en.txt"

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()
with open(textfile_en) as f:
    data_en = f.read().splitlines()

### 前処理

日本語、英語別々にトークナイザを作成する。

In [4]:
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"
pad_id = 3
vocab_size = 8000

In [ ]:
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

In [5]:
sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [6]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [7]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Decoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

`DataLoader`の作成。

In [8]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # Encoderへの入力
        x_dec = en[:-1] # Decoderへの入力
        y_dec = en[1:] # Decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

# example
x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

num of train data: 178487
num of test data: 44621


(torch.Size([32, 53]), torch.Size([32, 85]), torch.Size([32, 85]))


---

## Seq2Seqを用いた翻訳モデル

Encoder、Decoderを作成し、Seq2Seqモデルを作成する。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。LSTMと線形層で作る。

ここで、何を隠れ状態とするかを考える。Decoderに渡したい隠れ状態として満たしてほしい条件は以下である。

- （padを除いた）全ての入力を参照して出力されている
- 固定長

これらを全て満たしていれば何でもよい。

よくあるのは最後の隠れ状態。これが一番シンプルで実装も簡単。

ただこれだと学習時にpadトークンが隠れ状態に関与してしまう。多くの場合、推論時にpadトークンは存在しないため、学習時もpadトークンを考慮しない出力をさせた方が良さそう。また、padの数が多くなるにつれて隠れ状態がある一定の値に収束してしまう（経験談）。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうみたい。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、入力文を考慮した出力が行えない。

ではどうするかというと、「padを除いた最後の隠れ状態」とする。eosトークンが入力された時点の隠れ状態とも言える。これで上記の問題を解決できる。

他にもいくつかの案が考えられる。例えば、「padを除いた全ての隠れ状態の平均」とか。これも条件を満たす。

また、RNNを双方向にするという手法もある。Encoderは入力文の特徴を抽出できれば良いので、文の初めから順に処理しなければならない訳ではない。例えば逆から処理してもよい。

双方向RNNでは、順方向と逆方向の双方向で演算を行い、各時刻で二つの隠れ状態を出力する。両端の時刻の隠れ状態を取得することで二種類の隠れ状態が得られ、それらを足すないしは結合することで最終的な一つの隠れ状態が得られる。

双方向RNNは`torch.nn.RNN`の引数`bidirectional=True`で実装できる。ただ、各隠れ状態の出力がpadトークンを除いて行われて欲しいという望みがあり、それを実現するための実装が難しそうなので、今回は不採用。

結局どうしようかという感じだが、無難に「padを除いた最後の隠れ状態」を採用する。Encoderを以下のように実装する。

In [16]:
class Encoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_pos = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        skip = self.fc_skip(x)
        hs1, _ = self.lstm1(x) # (batch_size, seq_len, hidden_size)
        hs1 = hs1 + skip
        hs1 = self.dropout(hs1)
        hs2, _ = self.lstm2(hs1)
        hs2 = self.dropout(hs2)
        h1 = hs1[eos_pos] # (batch_size, hidden_size)
        h2 = hs2[eos_pos] # (batch_size, hidden_size)
        return h1, h2

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [17]:
class Decoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc):
        hc1, hc2 = hc
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        skip = self.fc_skip(x) # (batch_size, seq_len, hidden_size)
        hs, hc1 = self.lstm1(x, hc1) # (batch_size, seq_len, hidden_size)
        hs = hs + skip
        hs = self.dropout(hs)
        hs, hc2 = self.lstm2(hs, hc2)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, (hc1, hc2)

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = self.get_hc(h)
        y, _ = self.decoder(x_dec, hc)
        return y

    def get_hc(self, h):
        h1, h2 = h
        h1 = h1.unsqueeze(0)
        h2 = h2.unsqueeze(0)
        c1 = torch.zeros_like(h1)
        c2 = torch.zeros_like(h2)
        hc = ((h1, c1), (h2, c2))
        return hc

In [19]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 21,488,960



---

## 実践

### 学習

In [20]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [21]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [22]:
train(model, optimizer, n_epochs=20, prog_unit=1)

 1/20: #################### 100% [00:03:32.11] ppl train: 156.44, test: 104.08 
 2/20: #################### 100% [00:03:24.90] ppl train: 90.18, test: 80.36  
 3/20: #################### 100% [00:03:26.84] ppl train: 72.60, test: 68.94 
 4/20: #################### 100% [00:03:27.91] ppl train: 62.57, test: 61.82 
 5/20: #################### 100% [00:03:30.71] ppl train: 55.74, test: 57.06 
 6/20: #################### 100% [00:03:33.10] ppl train: 50.54, test: 53.48 
 7/20: #################### 100% [00:03:32.68] ppl train: 46.43, test: 50.73 
 8/20: #################### 100% [00:03:30.39] ppl train: 43.05, test: 48.79 
 9/20: #################### 100% [00:03:29.70] ppl train: 40.25, test: 47.13 
10/20: #################### 100% [00:03:31.25] ppl train: 37.86, test: 45.95 
11/20: #################### 100% [00:03:30.12] ppl train: 35.75, test: 44.94 
12/20: #################### 100% [00:03:30.12] ppl train: 33.92, test: 44.23 
13/20: #################### 100% [00:03:28.33] ppl train: 32.

In [23]:
model_path = "models/lm_seq2seq.pth"
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。決定的な出力にする。

In [24]:
def token_sampling(y, decisive=True):
    y = y.squeeze(0, 1)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    h_enc = model.encoder(in_ids)
    hc = model.get_hc(h_enc)
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).reshape(1, 1)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [25]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 時間が止まってしまったようでした
output: It was a little bit like a sliver.
answer: It was as though time had stopped.

input: 詳しく伝える時間はないのですが 非常に複雑な課題もこなし 間違えることを嫌います
output: I've never seen a lot of people, but I've got to tell you, I'm not going to be able to do anything about the things that I've been hearing about.
answer: She does very complex tasks, and I haven't got time to go into them, but the amazing thing about this female is she doesn't like making mistakes.

input: 本当に最期となったら どんな言葉を誰から 聞きたいですか?
output: What are you going to do with this person, who's a kid, who's a guy like, "What is your hand?"
answer: What do you want to hear at the very end, and from whom would you like to hear it?

input: 僕の名前は テイラー・ウィルソン
output: My name is Martin Luther King.
answer: So my name is Taylor Wilson.

input: よく考えないで作るから こういう事が起こる
output: You're thinking about it, and you're thinking, "This is a really good thing.
answer: It's letting stuff happen without thinking about it.



あまりよくないね。

訓練データに含まれていないものも試してみる。

In [26]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 非政府組織 企業 その他の市民団体が 他の人々の生活に影響を及ぼす強大な力を備えています
output: The most important social networking of social media is a community of people who are trying to help people who have access to the environment.
answer: They, businesses, other citizens' groups, have enormous power to affect the lives of our fellow human beings.

input: 真面目な話エネルギー省は フラッキングには関わっていません
output: So, the first thing I want to do is to make a difference, and I think that's a good thing.
answer: I mean seriously, the Department of Energy did not have anything to do with fracking.

input: 数万個の銀河系を調べることにより 発見した42個の超新星が 我々の頭上にある宇宙の理解を 覆したのならば 数十億の銀河系を調べることにより 42個の何倍ほどの超新星を得て 予測と全く一致しないようなものを 見出すことになるのでしょうか
output: We're going to see the next generation of planets, and we're going to see the Earth's genome, and we're going to see the next generation of planets, and we're going to see the problem of the Earth's atmosphere, and we're going to see the genomes of the Earth's surface, and we're going to see the next generation 

In [27]:
# original
sentences = [
    "ありがとう",
    "猫はかわいいのです",
    "上手く文章が書けるようになりました"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう
output: Thank you.

input: 猫はかわいいのです
output: They're not going to be able to get their hands.

input: 上手く文章が書けるようになりました
output: The first thing I did was I was working on the Internet.



びみょう。